### GO and KEGG enrichment analysis

In [ ]:
library(openxlsx)#读取.xlsx文件
library(ggplot2)#柱状图和点状图
library(stringr)#基因ID转换
library(enrichplot)#GO,KEGG,GSEA
library(clusterProfiler)#GO,KEGG,GSEA
library(GOplot)#弦图，弦表图，系统聚类图
library(DOSE)
library(ggnewscale)
library(topGO)#绘制通路网络图
library(circlize)#绘制富集分析圈图
library(ComplexHeatmap)#绘制图例
library(stringr)
library(ggplot2)
info <- read.xlsx( "gene_list.xlsx", rowNames = F,colNames = T)
info = read.csv("",header = T,row.names = 1)
GO_database <- 'org.Hs.eg.db' 
KEGG_database <- 'hsa' 
gene <- bitr(info$Gene,fromType = 'SYMBOL',toType = 'ENTREZID',OrgDb = GO_database)
geneIDselect <-select(org.Hs.eg.db, 
                      keys=info$Gene,
                      columns=c("ENTREZID","ENSEMBL","GENENAME"), 
                      keytype="SYMBOL") 
GO<-enrichGO( geneIDselect$ENTREZID,#
              OrgDb = GO_database,
              keyType = "ENTREZID",
              ont = "ALL",
              pvalueCutoff = 0.05,
              qvalueCutoff = 0.05,
              readable = T)
write.csv(data.frame(GO),"GO_ALL.csv")

KEGG<-enrichKEGG(gene$ENTREZID,
                 organism = KEGG_database,
                 pvalueCutoff = 0.05,
                 qvalueCutoff = 0.05)

library(DOSE)
KEGG2<-setReadable(KEGG, OrgDb = org.Hs.eg.db, keyType="ENTREZID")
write.csv(data.frame(KEGG2),"KEGG2_ALL.csv")

names(info) <- c('SYMBOL','Log2FoldChange','pvalue','padj')
info_merge <- merge(info,gene,by='SYMBOL')
GSEA_input <- info_merge$Log2FoldChange
names(GSEA_input) = info_merge$ENTREZID
GSEA_input = sort(GSEA_input, decreasing = TRUE)
GSEA_KEGG <- gseKEGG(GSEA_input, organism = KEGG_database, pvalueCutoff = 0.05)

b1 = barplot(GO, split="ONTOLOGY",font.size = 8,title="Enrichment GO")+facet_grid(ONTOLOGY~., scale="free")
dotplot(GO, split="ONTOLOGY",font.size = 8,title="Enrichment GO")+facet_grid(ONTOLOGY~., scale="free")
b2 = barplot(KEGG,showCategory = 20,title = 'Enrichment GO TOP20',font.size = 8)
dotplot(KEGG,showCategory = 20,title = 'Enrichment GO TOP20',font.size = 8)
ggsave("GO.ALL.pdf",b1,width = 7,height = 7)
ggsave("KEGG.20.pdf",b2,width = 7,height = 7)

genedata<-data.frame(ID=info$gene_symbol,logFC=info$log2FoldChange)
GOplotIn_BP<-GO[1:10,c(2,3,7,9)] 
GOplotIn_CC<-GO[2103:2112,c(2,3,7,9)]
GOplotIn_MF<-GO[2410:2419,c(2,3,7,9)]
GOplotIn_BP$geneID <-str_replace_all(GOplotIn_BP$geneID,'/',',') 
GOplotIn_CC$geneID <-str_replace_all(GOplotIn_CC$geneID,'/',',')
GOplotIn_MF$geneID <-str_replace_all(GOplotIn_MF$geneID,'/',',')
names(GOplotIn_BP)<-c('ID','Term','adj_pval','Genes')
names(GOplotIn_CC)<-c('ID','Term','adj_pval','Genes')
names(GOplotIn_MF)<-c('ID','Term','adj_pval','Genes')
GOplotIn_BP$Category = "BP"
GOplotIn_CC$Category = "CC"
GOplotIn_MF$Category = "MF"
circ_BP<-GOplot::circle_dat(GOplotIn_BP,genedata) 
circ_CC<-GOplot::circle_dat(GOplotIn_CC,genedata) 
circ_MF<-GOplot::circle_dat(GOplotIn_MF,genedata) 
chord_BP<-chord_dat(data = circ_BP,genes = genedata) 
chord_CC<-chord_dat(data = circ_CC,genes = genedata) 
chord_MF<-chord_dat(data = circ_MF,genes = genedata) 
GOChord(data = chord_BP,
        title = 'GO-Biological Process',space = 0.01,
        limit = c(1,1),gene.order = 'logFC',gene.space = 0.25,gene.size = 5,
        lfc.col = c('red','white','blue'), 
        process.label = 10) 
GOChord(data = chord_CC,title = 'GO-Cellular Component',space = 0.01,
        limit = c(1,1),gene.order = 'logFC',gene.space = 0.25,gene.size = 5,
        lfc.col = c('red','white','blue'), 
        process.label = 10) 
GOChord(data = chord_MF,title = 'GO-Mollecular Function',space = 0.01,
        limit = c(1,1),gene.order = 'logFC',gene.space = 0.25,gene.size = 5,
        lfc.col = c('red','white','blue'), 
        process.label = 10)

chord<-chord_dat(data = circ_BP,genes = genedata) 
GOCluster(circ_BP,GOplotIn_BP$Term) 
chord<-chord_dat(data = circ_CC,genes = genedata)
GOCluster(circ_CC,GOplotIn_CC$Term) 
chord<-chord_dat(data = circ_MF,genes = genedata) 
GOCluster(circ_MF,GOplotIn_MF$Term)
GOplotIn_BP<-GO[1:3,c(2,3,7,9)] 
GOplotIn_BP$geneID <-str_replace_all(GOplotIn_BP$geneID,'/',',')
names(GOplotIn_BP)<-c('ID','Term','adj_pval','Genes')
GOplotIn_BP$Category = "BP"
circ_BP<-GOplot::circle_dat(GOplotIn_BP,genedata)
chord<-chord_dat(data = circ_BP,genes = genedata) 
GOCluster(circ_BP,GOplotIn_BP$Term) 

GOplotIn_KEGG<- KEGG2[1:5,c(1,2,6,8)]
GOplotIn_KEGG$geneID <-str_replace_all(GOplotIn_KEGG$geneID,'/',',')
names(GOplotIn_KEGG)<-c('ID','Term','adj_pval','Genes')
GOplotIn_KEGG$Category = "KEGG"
circ_KEGG<-GOplot::circle_dat(GOplotIn_KEGG,genedata)
chord<-chord_dat(data = circ_KEGG,genes = genedata)

pdf("KEGG_sys.5.pdf",width = 10, height = 6)
GOCluster(circ_KEGG,GOplotIn_KEGG$Term) 
dev.off()

In [ ]:
### prognostic model

### Kaplan-meier analysis

In [ ]:
library(pheatmap)
library(survival)
library(tidyverse)
library(glmnet)  
library(survival)
library(survminer)
library(caret)
library(limma)
library(timeROC)
library(ROCR)
library(caret)
library(rms)
library(regplot)
library(tidyverse)
library(RColorBrewer)

In [ ]:
#####单因素Cox#####
uniq_cox  = function(pd_exp,x){
pFilter=0.05  
outTab=data.frame()
sigGenes=c("Status","OS_time")
for(i in colnames(pd_exp[,x:ncol(pd_exp)])){
  cox <- coxph(Surv(OS_time, Status) ~ pd_exp[,i], data = pd_exp)
  coxSummary = summary(cox)
  coxP=coxSummary$coefficients[,"Pr(>|z|)"]
  if(!is.na(coxP)){
      if(coxP<pFilter){
    sigGenes=c(sigGenes,i)
    outTab=rbind(outTab,
                 cbind(id=i,
                       HR=coxSummary$conf.int[,"exp(coef)"],
                       HR.95L=coxSummary$conf.int[,"lower .95"],
                       HR.95H=coxSummary$conf.int[,"upper .95"],
                       pvalue=coxSummary$coefficients[,"Pr(>|z|)"])
    )
  }
  }  
}
    
    return(list(outTab = outTab,uniq_gene = outTab$id))
}


### Cox and lasso algorithm constract model

In [ ]:
load("Split_data/GSE39582_pd_exp for lasso_cox.rdata")
uniq_cox_res = uniq_cox(pd_exp = train_pd_exp,7)

uniSigExp=rt[,sigGenes]
uniSigExp_data=cbind(id=row.names(uniSigExp),uniSigExp)
write.table(uniSigExp_data,file="UniSigExp.txt",sep="\t",row.names=F,quote=F)
rownames(outTab)=outTab$id
#####绘制单因素Cox森林图#####
rt=outTab[lassoGene,]
gene <- rownames(rt)
hr <- sprintf("%.3f",as.numeric(rt$"HR"))
hrLow  <- sprintf("%.3f",as.numeric(rt$"HR.95L"))
hrHigh <- sprintf("%.3f",as.numeric(rt$"HR.95H"))
Hazard.ratio <- paste0(hr,"(",hrLow,"-",hrHigh,")")
pVal <- ifelse(as.numeric(rt$pvalue)<0.001, "<0.001", sprintf("%.3f", as.numeric(rt$pvalue)))
#输出图形
pdf(file="unforest.pdf", width = 6,height = 6)
n <- nrow(rt)
nRow <- n+1
ylim <- c(1,nRow)
layout(matrix(c(1,2),nc=2),width=c(3,2))
xlim = c(0,3)
par(mar=c(4,2.5,2,1))
plot(1,xlim=xlim,ylim=ylim,type="n",axes=F,xlab="",ylab="")
text.cex=0.8
text(0,n:1,gene,adj=0,cex=text.cex)
text(1.5-0.5*0.2,n:1,pVal,adj=1,cex=text.cex);text(1.5-0.5*0.2,n+1,'pvalue',cex=text.cex,font=2,adj=1)
text(3,n:1,Hazard.ratio,adj=1,cex=text.cex);text(3,n+1,'Hazard ratio',cex=text.cex,font=2,adj=1,)
par(mar=c(4,1,2,1),mgp=c(2,0.5,0))
xlim = c((min(as.numeric(hrLow))-0.1),(max(as.numeric(hrHigh))+0.1))
plot(1,xlim=xlim,ylim=ylim,type="n",axes=F,ylab="",xaxs="i",xlab="Hazard ratio")
arrows(as.numeric(hrLow),n:1,as.numeric(hrHigh),n:1,angle=90,code=3,length=0.05,col="darkblue",lwd=2.5)
abline(v=1,col="black",lty=2,lwd=2)
boxcolor = ifelse(as.numeric(hr) > 1, 'red', 'green')
points(as.numeric(hr), n:1, pch = 15, col = boxcolor, cex=1.3)
axis(1)
dev.off()
y = Surv(train_pd_exp$OS_time,train_pd_exp$Status)
x = as.matrix(train_pd_exp[,uniq_cox_res$uniq_gene])
fit = glmnet(x,y,family = "cox")
pdf("lasso_fit.pdf",width = 4,height = 4)
plot(fit,xvar = "lambda")
dev.off()
pdf("lasso_cv.fit.pdf",width = 4,height = 4)
plot(fit.cv)
dev.off()
fit.cv = cv.glmnet(x,y,family = "cox",type.measure = "deviance",alpha =  0.7)#0-1
plot(fit.cv)
se_gene1 = NULL
#while(length(se_gene1)<10){
#cv_fit <- cv.glmnet(x=x, y=y) #cv.glmnet()就是一个调优参数过
fit.cv = cv.glmnet(x,y,family = "cox",type.measure = "deviance",alpha =  0.7)#0-1
plot(fit.cv)
#系数图
#fit <- glmnet(x=x1, y=y1) #glmnet是构建模型的
#plot(fit,xvar = "lambda")
#重新构建模型
model_lasso_min = glmnet(x,y,family = 'cox',lambda = fit.cv$lambda.min)
model_lasso_1se = glmnet(x,y,family = 'cox',lambda = fit.cv$lambda.1se)
min_gene1 =rownames(model_lasso_min$beta)[as.numeric(model_lasso_min$beta)!=0]
se_gene1 =rownames(model_lasso_1se$beta)[as.numeric(model_lasso_1se$beta)!=0]
min_gene1;length(min_gene1)
se_gene1;length(se_gene1)
#}

#pd_exp = train_pd_exp[1:2000]
predict = predict(model_lasso_min,newx = x,s = fit.cv$lambda.min);res = cbind(y,predict)
#min
pred_min <- prediction(res[,3], res[,2])
auc_min = performance(pred_min,"auc")@y.values[[1]]
perf_min <- performance(pred_min,"tpr","fpr")
plot(perf_min,colorize=FALSE, col="blue") 
lines(c(0,1),c(0,1),col = "gray", lty = 4 )
text(0.8,0.2, labels = paste0("AUC = ",round(auc_min,3)))

e_p_train_select = train_pd_exp[,c("OS_time","Status",min_gene1)]
cox <- coxph(Surv(OS_time, Status) ~ ., data = e_p_train_select)
cox=step(cox,direction = "backward")
summary(cox)
#_p_train_select
cox_gene = coxph(formula = Surv(OS_time, Status) ~.,data = e_p_train_select)

cox_gene_sum = summary(cox_gene)
sig_gene = rownames(cox_gene_sum$coefficients)[cox_gene_sum$coefficients[,5]<0.05]
save(cox_gene,train_pd_exp,mod_gene,tset_pd_exp,entire,external,file="cox_model.rdata")


mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
#绘制KM

train =  TCGA_exp_pd_;#train_pd_exp;#;tset_pd_exp;entire;external；TCGA_exp_pd_
pd_exp = TCGA_exp_pd_;#train_pd_exp;#;tset_pd_exp;entire;external；TCGA_exp_pd_
riskScore=predict(cox_gene,type="risk",newdata=pd_exp)
risk=as.vector(ifelse(riskScore>median(riskScore),"high","low"))
risk = data.frame(pd_exp,riskScore,risk)
risk = risk[is.finite(risk$riskScore),]
iMigor210_risk  = risk

#绘制KM曲线
fit <- survfit(Surv(OS_time, Status) ~ risk, data = risk)
g2 = ggsurvplot(fit,
          pval = TRUE, conf.int = TRUE,
          #risk.table = TRUE, # 添加风险表
          risk.table.col = "strata", # 根据分层更改风险表颜色
          linetype = "strata", # 根据分层更改线型
          #surv.median.line = "hv", # 同时显示垂直和水平参考线
          ggtheme = theme_bw(), # 更改ggplot2的主题
          palette =mypalette(22)[c(1,6)])#定义颜色

risk$Status  = as.factor(risk$Status)
risk$Group = ifelse(risk$Status == "1","Dead","alive")

library(ggpubr)
comparisons =list(c("alive","Dead"))
X= ggplot(risk,aes(x = Group,y=riskScore,fill = Group))+
geom_boxplot(show.legend = F,width = 0.5)+theme_bw()+
scale_fill_manual(values=c("#c0e2f5", "#f8eab3"))+
labs(subtitle = "Risk",x ="Risk" ,y = "RiskScore")+stat_compare_means(comparisons = comparisons)+  # 调整其他文本的字体大小
  theme(aspect.ratio = 1,
    text = element_text(size = 10),  # 调整所有文本的字体大小
    axis.text = element_text(size = 15),  # 调整坐标轴标签的字体大小
    axis.title = element_text(size = 15),  # 调整坐标轴标题的字体大小
    legend.title = element_text(size = 14),  # 调整图例标题的字体大小
    legend.text = element_text(size = 12)  # 调整图例文本的字体大小
  )
data = risk#train_risk;test_risk;entire;TCGA_risk;iMigor210_risk
data$Age = as.numeric(data$Age)
data = data[!is.na(data$Age),]
data$Gender_ = ifelse(data$Gender == "Male",1,ifelse(data$Gender == "male",'1',0))
data$Gender_ = as.numeric(data$Gender_)
data$Clinical_stage = as.numeric(data$Clinical_stage) 
data$Status = as.numeric(data$Status)
cox_ = coxph(Surv(OS_time,Status)~Age+Gender_+riskScore+Clinical_stage,data = data)
